In [202]:
import os

import numpy as np
from astropy.io import ascii as asc
from astropy.io import fits
import astropy.constants as c
import astropy.units as u
from astropy.time import Time
from astropy.modeling import fitting, models

from matplotlib import pyplot as plt
%matplotlib

#https://github.com/abostroem/utilities
import supernova
import spectroscopy as spec
from spectroscopy import apply_redshift as deredshift
import visualization

Using matplotlib backend: Qt5Agg


In [3]:
IRTF_DIR = '../data/spectra/IRTF/'
SOFI_DIR = '../data/spectra/SOFI'
LCO_DIR = '../data/spectra/lco/'
EFOSC_DIR = '../data/spectra/EFOSC/'

FIG_DIR = '../figures'
VEL_DATA_DIR = '../data/line_info'

DATA_DIR_13ej = '../../data/2013ej'
DATA_DIR_13by = '../../data/2013by'

In [4]:
visualization.set_bright_colors()

In [5]:
paschen_a = 18750.0
paschen_b = 12820.0
paschen_g = 10940.0
paschen_d = 10050.0

balmer_a = 6563.0

In [6]:
z = 0.0069 #15oz redshift

# Read in Spectra

### Read IRFT

In [183]:
#Date-obs: 2015-10-10
tbdata_irtf = asc.read(os.path.join(IRTF_DIR, 'A15oz_merge.txt'), names=['wave', 'flux', 'err', 'junk'])

wave,flux,err,junk
float64,float64,float64,float64
0.68849689,3.8235029e-15,3.4284355e-16,0.0


### Read SOFI

In [8]:
tbdata_sofi1_blue = fits.getdata(os.path.join(SOFI_DIR, 'asassn15oz_20150905_2457270.58657_1.fits'), 1) #wave, flux, err, skyback
tbdata_sofi1_red = fits.getdata(os.path.join(SOFI_DIR, 'asassn15oz_20150905_2457270.62701_1.fits'), 1)
tbdata_sofi2_blue = fits.getdata(os.path.join(SOFI_DIR, 'asassn15oz_20151005_2457300.50252_1.fits'), 1)
tbdata_sofi2_red = fits.getdata(os.path.join(SOFI_DIR, 'asassn15oz_20151005_2457300.54299_1.fits'), 1)

### Read optical data

In [9]:
#2015-09-05  LCO
ofile_optical_sofi1 = fits.open(os.path.join(LCO_DIR, 'asassn15oz_20150904_redblu_122216.314.fits'))
flux = ofile_optical_sofi1[0].data[0,0, :]
wl = spec.calc_wavelength(ofile_optical_sofi1[0].header, np.arange(1, len(flux)+1))
spec_optical_sofi1 = spec.spectrum1d(wl, flux, ofile_optical_sofi1[0].data[3,0, :])
#2015-10-05 EFOSC
ofile_optical_sofi2 = fits.open(os.path.join(EFOSC_DIR, 'tASASSN-15oz_20151003_Gr13_Free_slit1.0_57720_1_e.fits'))
flux = ofile_optical_sofi2[0].data[0,0,:]
wl = spec.calc_wavelength(ofile_optical_sofi2[0].header, np.arange(1, len(flux)+1))
spec_optical_sofi2 = spec.spectrum1d(wl, flux, ofile_optical_sofi2[0].data[3,0,:])
#2015-10-10  LCO? 10/6 and 10/14

### Get Expected Velocities

In [10]:
def calc_velocity(obs_wl, rest_wl):
    velocity = c.c*(obs_wl/rest_wl - 1)
    return velocity

In [11]:
HA = 6563.0
HB = 4861.0
SiII = 6355.0
FeII = 5169.0
IR_dates = Time(['2015-09-05','2015-10-05', '2015-10-10'])

In [13]:
sn15oz = supernova.LightCurve2('asassn-15oz')
texpl = Time(sn15oz.jdexpl, format='jd')

In [14]:
new_fit_cachito = asc.read(os.path.join(VEL_DATA_DIR, 'cachito.tab'))
phase_cachito = (Time(new_fit_cachito['date'])-texpl).value
velocity_cachito = -1*calc_velocity(new_fit_cachito['vel0'], HA).to(u.km/u.s).value

new_fit_HA = asc.read(os.path.join(VEL_DATA_DIR, 'HA.tab'))
phase_HA = (Time(new_fit_HA['date'])-texpl).value
velocity_HA = -1*calc_velocity(new_fit_HA['vel0'], HA).to(u.km/u.s).value

new_fit_HB = asc.read(os.path.join(VEL_DATA_DIR, 'HB.tab'))
phase_HB = (Time(new_fit_HB['date'])-texpl).value
velocity_HB = -1*calc_velocity(new_fit_HB['vel0'], HB).to(u.km/u.s).value

tbdata_feII = asc.read(os.path.join(VEL_DATA_DIR, 'FeII_multi.tab'))
phase_feII = (Time(tbdata_feII['date'])-texpl).value
velocity_feII = -1*calc_velocity(tbdata_feII['vel0'], FeII).to(u.km/u.s).value



In [15]:
fitter_power = fitting.LevMarLSQFitter()
power_model = models.PowerLaw1D()

power_fit_cachito = fitter_power(power_model, phase_cachito, velocity_cachito)
power_fit_HA = fitter_power(power_model, phase_HA, velocity_HA)

power_fit_HB = fitter_power(power_model, phase_HB, velocity_HB)


#I had trouble fitting without good starting parameters, cachito looks like FeII, especially at early times
power_model_feII = models.PowerLaw1D(alpha=power_fit_cachito.alpha, x_0=power_fit_cachito.x_0) 
power_fit_feII = fitter_power(power_model_feII, phase_feII, velocity_feII)

balmer_velocity = power_fit_HA((IR_dates-texpl).value)
HB_velocity = power_fit_HB((IR_dates-texpl).value)
hv_hydrogen_velocity = power_fit_cachito((IR_dates-texpl).value)
si_velocity = power_fit_feII((IR_dates-texpl).value)

plt.plot(phase_HB, velocity_HB, 'o')
plt.plot(np.arange(1, 90), power_fit_HB(np.arange(1, 90)))

In [16]:
def calc_observed_wave(rest_wave, velocity):
    '''
    Assumes velocity given in km/s
    '''
    velocity = velocity*u.km/u.s
    shifted_wave = (velocity/c.c+1.0)*rest_wave
    return shifted_wave

In [166]:
fig = plt.figure(figsize=[10, 5])
ax_optical_SOFI1 = fig.add_subplot(3,2,1)
ax_ir_SOFI1 = fig.add_subplot(3,2,2)
ax_optical_SOFI2 = fig.add_subplot(3,2,3)
ax_ir_SOFI2 = fig.add_subplot(3,2,4)
ax_optical_IRTF = fig.add_subplot(3,2,5)
ax_ir_IRTF = fig.add_subplot(3,2,6)

#Plot First SOFI spec deredshifted
l1, = ax_ir_SOFI1.plot(deredshift(tbdata_sofi1_blue['WAVE'][0], z), tbdata_sofi1_blue['FLUX'][0], label='SOFI-2015-09-05')
ax_ir_SOFI1.plot(deredshift(tbdata_sofi1_red['WAVE'][0], z), tbdata_sofi1_red['FLUX'][0], color=l1.get_color())
ax_ir_SOFI1.set_ylim(0.4E-15, 2.5E-15)
#and corresponding optical spectrum
ax_optical_SOFI1.plot(deredshift(spec_optical_sofi1.wave, z), spec_optical_sofi1.flux, color=l1.get_color())

#Plot Second SOFI spec deredshifted
l2, = ax_ir_SOFI2.plot(deredshift(tbdata_sofi2_blue['WAVE'][0], z), tbdata_sofi2_blue['FLUX'][0], label='SOFI-2015-10-05')
ax_ir_SOFI2.plot(deredshift(tbdata_sofi2_red['WAVE'][0], z), tbdata_sofi2_red['FLUX'][0], color=l2.get_color())
ax_ir_SOFI2.set_ylim(0.4E-15, 1.25E-15)
#and corresponding optical spectrum
ax_optical_SOFI2.plot(deredshift(spec_optical_sofi2.wave, z), spec_optical_sofi2.flux, color=l2.get_color())


#Plot ITRF spec deredshifted
l3, = ax_ir_IRTF.plot(deredshift(tbdata_irtf['wave']*10**4, z), tbdata_irtf['flux'], label='IRTF - 2015-10-10')
ax_ir_IRTF.set_ylim(0.4E-15, 2E-15)
#and corresponding optical spectrum
ax_optical_IRTF.plot(deredshift(spec_optical_sofi2.wave, z), spec_optical_sofi2.flux, color=l2.get_color())

for ax_ir in [ax_ir_SOFI1, ax_ir_SOFI2, ax_ir_IRTF]:
    ax_ir.set_xlim(9000, 13500)

for ax_optical in [ax_optical_SOFI1, ax_optical_SOFI2, ax_optical_IRTF]:
    ax_optical.set_xlim(4500, 7000)
    ax_optical.set_ylim(0.25E-14,1.0E-14)


# Compare to 2013by and 2013ej

In [18]:
ofile_2013ej_blue = fits.open(os.path.join(DATA_DIR_13ej, '2013ej_20130829_2456533.70935_1.fits'))
ofile_2013ej_red = fits.open(os.path.join(DATA_DIR_13ej, '2013ej_20130829_2456533.72646_1.fits'))

ofile_2013by = fits.open(os.path.join(DATA_DIR_13by, '2013by_20130522_2456435.008_1.fits'))

In [19]:
z_2013ej = 0.002192
z_2013by = 0.00382

sn13ej = supernova.LightCurve2('2013ej')
sn13by = supernova.LightCurve2('2013by')

In [20]:
ofile_2013ej_blue[1].data['wave'][0]

array([  9381.96777344,   9388.90429688,   9395.84179688, ...,
        16464.875     ,  16471.81054688,  16478.74804688], dtype=float32)

In [165]:
fig = plt.figure(figsize=[15, 5])
ax_ir_SOFI2_blue = fig.add_subplot(1,2,1)
ax_ir_SOFI2_red =  fig.add_subplot(1,2,2)



#Plot Second SOFI spec deredshifted
ax_ir_SOFI2_blue.plot(deredshift(tbdata_sofi2_blue['WAVE'][0], z), tbdata_sofi2_blue['FLUX'][0], label='15oz: SOFI-{}d'.format(Time('2015-10-05')-Time(sn15oz.jdexpl, format='jd')))
ax_ir_SOFI2_blue.plot(deredshift(ofile_2013ej_blue[1].data['wave'][0], z_2013ej), 
                      ofile_2013ej_blue[1].data['FLUX'][0]/5, 
                      label='13ej: SOFI-{}'.format(Time(ofile_2013ej_blue[0].header['DATE-OBS'])-Time(sn13ej.jdexpl, format='jd')))

ax_ir_SOFI2_blue.plot(deredshift(ofile_2013by[1].data['wave'][0], z_2013by), 
                      ofile_2013by[1].data['FLUX'][0]/10-0.5E-15,
                      label='13by: FIRE-{}'.format(Time(ofile_2013by[0].header['DATE-OBS'])-Time(sn13by.jdexpl, format='jd')))


ax_ir_SOFI2_red.plot(deredshift(tbdata_sofi2_red['WAVE'][0], z), tbdata_sofi2_red['FLUX'][0])
ax_ir_SOFI2_red.plot(deredshift(ofile_2013ej_red[1].data['wave'][0], z_2013ej), 
                     ofile_2013ej_red[1].data['FLUX'][0]/5, 
                     label='13ej: SOFI-{}'.format(Time(ofile_2013ej_red[0].header['DATE-OBS'])-Time(sn13ej.jdexpl, format='jd')))

ax_ir_SOFI2_red.plot(deredshift(ofile_2013by[1].data['wave'][0], z_2013by), 
                      ofile_2013by[1].data['FLUX'][0]/10,
                      label='13by: FIRE-{}'.format(Time(ofile_2013by[0].header['DATE-OBS'])-Time(sn13by.jdexpl, format='jd')))

ax_ir_SOFI2_blue.set_ylim(0.2E-15, 5E-15)
ax_ir_SOFI2_red.set_ylim(0.0E-15, 0.12E-14)
ax_ir_SOFI2_blue.set_xlim(9000, 17000)
ax_ir_SOFI2_red.set_xlim(15000, 25000)
ax_ir_SOFI2_blue.legend()

ax_ir_SOFI2_blue.annotate(
        r'Pa-$\beta$',
        xy=(12500, 1.5E-15), xytext=(0, 50), 
        arrowprops=dict(arrowstyle='-'),
        rotation='vertical', 
        xycoords='data', 
        textcoords='offset points', 
        ha='center')

ax_ir_SOFI2_blue.annotate(
        r'Pa-$\gamma$',
        xy=(10600, 2E-15), xytext=(0, 50), 
        arrowprops=dict(arrowstyle='-'),
        rotation='vertical', 
        xycoords='data', 
        textcoords='offset points', 
        ha='center')

ax_ir_SOFI2_blue.annotate(
        r'Pa-$\delta$',
        xy=(9850, 3E-15), xytext=(0, 50), 
        arrowprops=dict(arrowstyle='-'),
        rotation='vertical', 
        xycoords='data', 
        textcoords='offset points', 
        ha='center')

Text(0,50,'Pa-$\\delta$')

# ID Lines

In [144]:
def plot_element(element, rest_wave, velocity, ax, color=None, linestyle='-', alpha=1.0):
    rest_wave = np.array(rest_wave)
    obs_wave = calc_observed_wave(rest_wave, velocity)
    ylim = ax.get_ylim()
    ax.vlines(obs_wave, ymin=ylim[0], ymax=ylim[1], label='{}@{}km/s'.format(element, velocity), color=color, linestyle=linestyle, alpha=alpha)
    print(ylim)
    return ax
    

In [145]:
HI_Pasch = [10500, 10940, 12820, 18750] #
HI_Bra = [14580, 18170, 19440, 21660, 26250] #
HI = [16440, 16810, 18180, 19450] #
HeI = [10830, 20587] #
MgI = [9415.,9256.,9246.5, 11830, 15040, 15750, 17110] #
MgII = [10092]#
SrII = [10327] #
SI = [10460, 22570] #
OI = [8222.,8446.,9266.,11290, 13170] #
OII = [13811.,21085.]#
SiI = [12030,15893] #
NaI = [22060] #
CaI = [19450, 19850] #
CaII = [9906, 11836, 11947] #
FeII = [12570, 13210, 16444] #
CI = [10691,12614.,13502.,16600.,16890.] #
CII = [9903.,17846., 18905.] #



In [234]:
plt.close()
fig = plt.figure(figsize=[15, 5])
ax_ir_SOFI2_blue = fig.add_subplot(1,2,1)
ax_ir_SOFI2_red =  fig.add_subplot(1,2,2)



#Plot Second SOFI spec deredshifted
ax_ir_SOFI2_blue.plot(deredshift(tbdata_sofi2_blue['WAVE'][0], z), tbdata_sofi2_blue['FLUX'][0], label='15oz: SOFI-{}d'.format(Time('2015-10-05')-Time(sn15oz.jdexpl, format='jd')))

ax_ir_SOFI2_red.plot(deredshift(tbdata_sofi2_red['WAVE'][0], z), tbdata_sofi2_red['FLUX'][0])


ax_ir_SOFI2_blue.set_ylim(0.2E-15, 1.5E-15)
ax_ir_SOFI2_red.set_ylim(0.0E-15, 2E-16)
ax_ir_SOFI2_blue.set_xlim(9000, 15000)
ax_ir_SOFI2_red.set_xlim(15000, 25000)

#ax_ir_SOFI2_blue.annotate(
#        r'Pa-$\beta$',
#        xy=(12500, 0.8E-15), xytext=(0, 50), 
#        arrowprops=dict(arrowstyle='-'),
#        rotation='vertical', 
#        xycoords='data', 
#        textcoords='offset points', 
#        ha='center')
#
#ax_ir_SOFI2_blue.annotate(
#        r'Pa-$\gamma$',
#        xy=(10600, 1E-15), xytext=(0, 50), 
#        arrowprops=dict(arrowstyle='-'),
#        rotation='vertical', 
#        xycoords='data', 
#        textcoords='offset points', 
#        ha='center')
#
#ax_ir_SOFI2_blue.annotate(
#        r'Pa-$\delta$',
#        xy=(10200, 1.1E-15), xytext=(0, 50), 
#        arrowprops=dict(arrowstyle='-'),
#        rotation='vertical', 
#        xycoords='data', 
#        textcoords='offset points', 
#        ha='center')

ax_ir_SOFI2_blue = plot_element('HeI', HeI,         -8000, ax_ir_SOFI2_blue, color='r')
ax_ir_SOFI2_red = plot_element('HeI', HeI,          -8000, ax_ir_SOFI2_red, color='r')

ax_ir_SOFI2_blue = plot_element('HI Bra', HI_Bra,   -2500, ax_ir_SOFI2_blue, color='Gray')
ax_ir_SOFI2_red = plot_element('HI Bra', HI_Bra,    -2500, ax_ir_SOFI2_red, color='Gray')

ax_ir_SOFI2_blue = plot_element('CaII', CaII,       -7000, ax_ir_SOFI2_blue, color='g')
ax_ir_SOFI2_red = plot_element('CaII', CaII,        -7000, ax_ir_SOFI2_red, color='g')

ax_ir_SOFI2_blue = plot_element('CI', CI,           -5000, ax_ir_SOFI2_blue, color='lime')
ax_ir_SOFI2_red = plot_element('CI', CI,            -5000, ax_ir_SOFI2_red, color='lime')

ax_ir_SOFI2_blue = plot_element('OI', OI,           -6000, ax_ir_SOFI2_blue, color='gold')
ax_ir_SOFI2_red = plot_element('OI', OI,            -6000, ax_ir_SOFI2_red, color='gold')

ax_ir_SOFI2_blue = plot_element('SiI', SiI,         -5000, ax_ir_SOFI2_blue, color='purple')
ax_ir_SOFI2_red = plot_element('SiI', SiI,          -5000, ax_ir_SOFI2_red, color='purple')

ax_ir_SOFI2_blue = plot_element('HI Pasch', HI_Pasch, -8000, ax_ir_SOFI2_blue, color='k')
ax_ir_SOFI2_red = plot_element('HI Pasch', HI_Pasch,  -8000, ax_ir_SOFI2_red, color='k')

ax_ir_SOFI2_blue = plot_element('MgII', MgII,         -7000, ax_ir_SOFI2_blue, color='c')
ax_ir_SOFI2_red = plot_element('MgII', MgII,          -7000, ax_ir_SOFI2_red, color='c')

ax_ir_SOFI2_blue = plot_element('MgI', MgI,         -7000, ax_ir_SOFI2_blue, color='c', linestyle='--')
ax_ir_SOFI2_red = plot_element('MgI', MgI,          -7000, ax_ir_SOFI2_red, color='c', linestyle='--')

#ax_ir_SOFI2_blue = plot_element('SrII', SrII,       -5000, ax_ir_SOFI2_blue, color='orange', linestyle=':', alpha=0.5)
#ax_ir_SOFI2_red = plot_element('SrII', SrII,        -5000, ax_ir_SOFI2_red, color='orange',  linestyle=':', alpha=0.5)
#
#ax_ir_SOFI2_blue = plot_element('SI', SI,           -7000, ax_ir_SOFI2_blue, color='Navy',   linestyle=':', alpha=0.5)
#ax_ir_SOFI2_red = plot_element('SI', SI,            -7000, ax_ir_SOFI2_red, color='Navy',    linestyle=':', alpha=0.5)
#
#ax_ir_SOFI2_red = plot_element('FeII', FeII,        -5000, ax_ir_SOFI2_red, color='r',       linestyle=':', alpha=0.5)
#ax_ir_SOFI2_blue = plot_element('FeII', FeII,       -5000, ax_ir_SOFI2_blue, color='r',      linestyle=':', alpha=0.5)
#
#ax_ir_SOFI2_blue = plot_element('CaI', CaI,         -5000, ax_ir_SOFI2_blue, color='g', linestyle=':', alpha=0.5)
#ax_ir_SOFI2_red = plot_element('CaI', CaI,          -5000, ax_ir_SOFI2_red, color='g',  linestyle=':', alpha=0.5)
#
#ax_ir_SOFI2_blue = plot_element('NaI', NaI,         -5000, ax_ir_SOFI2_blue, color='m',      linestyle=':', alpha=0.5)
#ax_ir_SOFI2_red = plot_element('NaI', NaI,          -5000, ax_ir_SOFI2_red, color='m',       linestyle=':', alpha=0.5)
#
#ax_ir_SOFI2_blue = plot_element('OII', OII,           -6000, ax_ir_SOFI2_blue, color='gold', linestyle=':', alpha=0.5)
#ax_ir_SOFI2_red = plot_element('OII', OII,            -6000, ax_ir_SOFI2_red, color='gold', linestyle=':', alpha=0.5)
#
#ax_ir_SOFI2_blue = plot_element('CII', CII,           -7000, ax_ir_SOFI2_blue, color='lime', linestyle=':', alpha=0.5)
#ax_ir_SOFI2_red = plot_element('CII', CII,            -7000, ax_ir_SOFI2_red, color='lime', linestyle=':', alpha=0.5)

ax_ir_SOFI2_blue.legend(ncol=2, fontsize='x-small')

(2e-16, 1.4999999999999999e-15)
(0.0, 2e-16)
(2e-16, 1.4999999999999999e-15)
(0.0, 2e-16)
(2e-16, 1.4999999999999999e-15)
(0.0, 2e-16)
(2e-16, 1.4999999999999999e-15)
(0.0, 2e-16)
(2e-16, 1.4999999999999999e-15)
(0.0, 2e-16)
(2e-16, 1.4999999999999999e-15)
(0.0, 2e-16)
(2e-16, 1.4999999999999999e-15)
(0.0, 2e-16)
(2e-16, 1.4999999999999999e-15)
(0.0, 2e-16)
(2e-16, 1.4999999999999999e-15)
(0.0, 2e-16)


# Make Figure

In [282]:
import matplotlib as mpl
font = mpl.font_manager.FontProperties()
font.set_family('cursive')
font.set_name(mpl.rcParams['font.monospace'][6:])
font.set_size('x-small')
print(font.get_name())
print(mpl.rcParams['font.monospace'][6:])

DejaVu Sans Mono
['Courier', 'Fixed', 'Terminal', 'monospace']


## Two Plot Form

In [348]:
fig = plt.figure(figsize=[15, 5])
ax_ir_blue = fig.add_subplot(1,2,1)
ax_ir_red =  fig.add_subplot(1,2,2)

telluric_blue = (13300, 14500)
telluric_red = (17800, 19250)

rest_irtf_wave = deredshift(tbdata_irtf['wave']*10**4, z)

segment1_indx = rest_irtf_wave<telluric_blue[0]
segment2_indx = (rest_irtf_wave>telluric_blue[1]) & (rest_irtf_wave<telluric_red[0])
segment3_indx = rest_irtf_wave>telluric_red[1]

ax_ir_blue.axvspan(telluric_blue[0], telluric_blue[1], color='LightGray', alpha=0.5)
ax_ir_red.axvspan(telluric_red[0], telluric_red[1], color='LightGray', alpha=0.5)
#Plot Second SOFI spec deredshifted
l2, = ax_ir_blue.plot(deredshift(tbdata_sofi1_blue['WAVE'][0], z), tbdata_sofi1_blue['FLUX'][0]+0.8E-15, label='15oz: SOFI-{}d'.format(int((Time('2015-09-05')-Time(sn15oz.jdexpl, format='jd')).value)))
l3, = ax_ir_blue.plot(deredshift(tbdata_sofi2_blue['WAVE'][0], z), tbdata_sofi2_blue['FLUX'][0]+0.6E-15, label='15oz: SOFI-{}d'.format(int((Time('2015-10-05')-Time(sn15oz.jdexpl, format='jd')).value)))
l1, = ax_ir_blue.plot(rest_irtf_wave[segment1_indx], tbdata_irtf['flux'][segment1_indx], label='15oz: IRTF-{}d'.format(int((Time('2015-10-10')-Time(sn15oz.jdexpl, format='jd')).value)))
ax_ir_blue.plot(rest_irtf_wave[segment2_indx], tbdata_irtf['flux'][segment2_indx], color=l1.get_color(), label=None)





ax_ir_red.plot(deredshift(tbdata_sofi1_red['WAVE'][0], z), tbdata_sofi1_red['FLUX'][0]+0.4E-15)
ax_ir_red.plot(deredshift(tbdata_sofi2_red['WAVE'][0], z), tbdata_sofi2_red['FLUX'][0]+0.35E-15)
ax_ir_red.plot(rest_irtf_wave[segment2_indx], tbdata_irtf['flux'][segment2_indx], color=l1.get_color())
ax_ir_red.plot(rest_irtf_wave[segment3_indx], tbdata_irtf['flux'][segment3_indx], color=l1.get_color())

ax_ir_blue.set_ylim(0.2E-15, 3.5E-15)
ax_ir_red.set_ylim(1E-16, 6.75E-16)
ax_ir_blue.set_xlim(9000, 16500)
ax_ir_red.set_xlim(14800, 24900)
#ax_ir_blue.legend([l2, l3, l1], [l2.get_label(), l3.get_label(), l1.get_label()])

annotation_dict_blue = {'CaII 9906': (9686,       2.75E-15),
                   'MgII 10092': (9857,           2.75E-15),
                   r'P-$\delta$':(10232,          2.5E-15),
                   'CI 10691/ HeI 10830': (10500, 2.25E-15),
                   r'P-$\gamma$': (10675,         2.25E-15),
                   'OI 11290': (11075,            2.25E-15),
                   'CaII 11836': (11572,          1.75E-15),
                   'CaII 11947': (11685,          1.75E-15),
                   r'P-$\beta$':(12503,           1.75E-15),
                    'SiI 15893': (15632,          1.25E-15)}

for key in annotation_dict_blue.keys():
    x,y = annotation_dict_blue[key]
    yoffset=0.1E-15
    textoffset = 0.05E-15
    ax_ir_blue.axvline(x, color='k', alpha=0.05, zorder=0)
    ax_ir_blue.plot([x,x], [y, y+yoffset], color='k')
    ax_ir_blue.text(x, y+yoffset+textoffset, key, rotation='vertical', ha='center', va='bottom',
        fontproperties=font)
    
    
annotation_dict_red_above = {r'Br-$\gamma$':(21480, 5.2E-16),
                       'HeI 20587': (20041,   5.4E-16)}
annotation_dict_red_below = {'SiI 15893': (15632,   3.5E-16),
                       'CI 16600': (16351,    3.25E-16), 
                       'CI 16890': (16655,    3.0E-16)}
for key in annotation_dict_red_above.keys():
    x,y = annotation_dict_red_above[key]
    yoffset=0.03E-15
    textoffset = 0.02E-15
    ax_ir_red.axvline(x, color='k', alpha=0.05, zorder=0)
    ax_ir_red.plot([x,x], [y, y+yoffset], color='k')
    ax_ir_red.text(x, y+yoffset+textoffset, key, rotation='vertical', ha='center', va='bottom', 
        fontproperties=font)
    
for key in annotation_dict_red_below.keys():
    x,y = annotation_dict_red_below[key]
    yoffset=0.03E-15
    textoffset = 0.02E-15
    ax_ir_red.axvline(x, color='k', alpha=0.05, zorder=0)
    ax_ir_red.plot([x,x], [y, y-yoffset], color='k')
    ax_ir_red.text(x, y-yoffset-textoffset, key, rotation='vertical', ha='center', va='top', 
        fontproperties=font)
ax_ir_blue.set_ylabel('Flux ($ergs/cm^2/s$)+ offset')
ax_ir_blue.set_xlabel('Wavelength ($\AA$)')
ax_ir_red.set_xlabel('Wavelength ($\AA$)')

ax_ir_blue.text(16100, 2.0E-15, '8d')
ax_ir_blue.text(16100, 1.4E-15, '38d')
ax_ir_blue.text(16100, 0.5E-15, '43d')
plt.savefig(os.path.join(FIG_DIR, 'ir_spec_montage_two_plots.pdf'))

## One Plot Form

In [314]:
fig = plt.figure(figsize=[15, 5])
ax_ir = fig.add_subplot(1,1,1)

telluric_blue = (13300, 14500)
telluric_red = (17800, 19250)

rest_irtf_wave = deredshift(tbdata_irtf['wave']*10**4, z)

segment1_indx = rest_irtf_wave<telluric_blue[0]
segment2_indx = (rest_irtf_wave>telluric_blue[1]) & (rest_irtf_wave<telluric_red[0])
segment3_indx = rest_irtf_wave>telluric_red[1]

ax_ir.axvspan(telluric_blue[0], telluric_blue[1], color='LightGray', alpha=0.5)
ax_ir.axvspan(telluric_red[0], telluric_red[1], color='LightGray', alpha=0.5)
#Plot Second SOFI spec deredshifted
l2, = ax_ir.plot(deredshift(tbdata_sofi1_blue['WAVE'][0], z), tbdata_sofi1_blue['FLUX'][0]+1.5E-15)
l3, = ax_ir.plot(deredshift(tbdata_sofi2_blue['WAVE'][0], z), tbdata_sofi2_blue['FLUX'][0]+1E-15)
l1, = ax_ir.plot(rest_irtf_wave[segment1_indx], tbdata_irtf['flux'][segment1_indx])
ax_ir.plot(rest_irtf_wave[segment2_indx], tbdata_irtf['flux'][segment2_indx], color=l1.get_color())





ax_ir.plot(deredshift(tbdata_sofi1_red['WAVE'][0], z), tbdata_sofi1_red['FLUX'][0]+1.5E-15, color=l2.get_color())
ax_ir.plot(deredshift(tbdata_sofi2_red['WAVE'][0], z), tbdata_sofi2_red['FLUX'][0]+1E-15, color=l3.get_color())
ax_ir.plot(rest_irtf_wave[segment2_indx], tbdata_irtf['flux'][segment2_indx], color=l1.get_color())
ax_ir.plot(rest_irtf_wave[segment3_indx], tbdata_irtf['flux'][segment3_indx], color=l1.get_color())

ax_ir.set_ylim(0.0E-15, 5E-15)
ax_ir.set_xlim(9000, 24900)

annotation_dict_blue = {'CaII 9906': (9686, 3.5E-15),
                   'MgII 10092': (9857, 3.5E-15),
                   r'P-$\delta$':(10232, 3.1E-15),
                   'CI 10691/ HeI 10830': (10500, 3E-15),
                   r'P-$\gamma$': (10675, 3E-15),
                   'OI 11290': (11075, 3E-15),
                   'CaII 11836': (11572, 2.5E-15),
                   'CaII 11947': (11685, 2.5E-15),
                   r'P-$\beta$':(12503, 2.5E-15)}

for key in annotation_dict_blue.keys():
    x,y = annotation_dict_blue[key]
    yoffset=0.1E-15
    textoffset = 0.05E-15
    ax_ir.axvline(x, color='k', alpha=0.05, zorder=0)
    ax_ir.plot([x,x], [y, y+yoffset], color='k')
    ax_ir.text(x, y+yoffset+textoffset, key, rotation='vertical', ha='center', va='bottom',
        fontproperties=font)
    
    
annotation_dict_red = {'SiI 15893': (15632,   2.0E-15),
                       'CI 16600': (16351,    2.0E-15), 
                       'CI 16890': (16655,    2.0E-15),
                       r'Br-$\gamma$':(21480, 1.8E-15),
                       'HeI 20587': (20041,   1.8E-15)}
for key in annotation_dict_red.keys():
    x,y = annotation_dict_red[key]
    yoffset=0.1E-15
    textoffset = 0.05E-15
    ax_ir.axvline(x, color='k', alpha=0.05, zorder=0)
    ax_ir.plot([x,x], [y, y+yoffset], color='k')
    ax_ir.text(x, y+yoffset+textoffset, key, rotation='vertical', ha='center', va='bottom', 
        fontproperties=font)
ax_ir.set_ylabel('Flux ($ergs/cm^2/s$)+ offset')
ax_ir.set_xlabel('Wavelength ($\AA$)')
ax_ir.set_xlabel('Wavelength ($\AA$)')

ax_ir.text(24500, 1.7E-15, '8d')
ax_ir.text(24500, 1.2E-15, '38d')
ax_ir.text(24500, 0.3E-15, '43d')


Text(24500,3e-16,'43d')

# Log Space

In [362]:
spec_sofi1_blue = spec.spectrum1d(deredshift(tbdata_sofi1_blue['WAVE'][0],z), 
                                  tbdata_sofi1_blue['FLUX'][0], 
                                  tbdata_sofi1_blue['ERR'][0])
spec_sofi1_red = spec.spectrum1d( deredshift(tbdata_sofi1_red['WAVE'][0], z), 
                                  tbdata_sofi1_red[ 'FLUX'][0], 
                                  tbdata_sofi1_red[ 'ERR'][0])
spec_sofi2_blue = spec.spectrum1d(deredshift(tbdata_sofi2_blue['WAVE'][0], z), 
                                  tbdata_sofi2_blue['FLUX'][0], 
                                  tbdata_sofi2_blue['ERR'][0])
spec_sofi2_red = spec.spectrum1d( deredshift(tbdata_sofi2_red['WAVE'][0], z), 
                                 tbdata_sofi2_red[ 'FLUX'][0], 
                                 tbdata_sofi2_red[ 'ERR'][0])
spec_irtf = spec.spectrum1d(deredshift(tbdata_irtf['wave']*10**4, z), 
                            tbdata_irtf['flux'], 
                            tbdata_irtf['err'])

In [424]:
scale_sofi1_blue = spec.scale_spectra(spec_sofi1_blue, spec_sofi1_blue, wlmin=15000, wlmax=16000)
scale_sofi1_red = spec.scale_spectra(spec_sofi1_red, spec_sofi1_blue,   wlmin=15000, wlmax=16000)
scale_sofi2_blue = spec.scale_spectra(spec_sofi2_blue, spec_sofi1_blue, wlmin=15000, wlmax=16000)
scale_sofi2_red = spec.scale_spectra(spec_sofi2_red, spec_sofi1_blue,   wlmin=15000, wlmax=16000)
scale_irtf = spec.scale_spectra(spec_irtf, spec_sofi1_blue,             wlmin=15000, wlmax=16000)

In [426]:
fig = plt.figure(figsize=[10, 5])
ax_ir = fig.add_subplot(1,1,1)

telluric_blue = (13300, 14500)
telluric_red = (17800, 19250)


segment1_indx = scale_irtf.wave<telluric_blue[0]
segment2_indx = (scale_irtf.wave>telluric_blue[1]) & (scale_irtf.wave<telluric_red[0])
segment3_indx = scale_irtf.wave>telluric_red[1]

ax_ir.axvspan(telluric_blue[0], telluric_blue[1], color='LightGray', alpha=0.5)
ax_ir.axvspan(telluric_red[0], telluric_red[1], color='LightGray', alpha=0.5)
#Plot Second SOFI spec deredshifted
l2, = ax_ir.plot(scale_sofi1_blue.wave, np.log10(scale_sofi1_blue.flux))
l3, = ax_ir.plot(scale_sofi2_blue.wave, np.log10(scale_sofi2_blue.flux)-0.6)
l1, = ax_ir.plot(scale_irtf.wave[segment1_indx], np.log10(scale_irtf.flux[segment1_indx])-1)
ax_ir.plot(scale_irtf.wave[segment2_indx], np.log10(scale_irtf.flux[segment2_indx])-1, color=l1.get_color())


ax_ir.plot(scale_sofi1_red.wave, np.log10(scale_sofi1_red.flux), color=l2.get_color())
ax_ir.plot(scale_sofi2_red.wave, np.log10(scale_sofi2_red.flux)-0.6, color=l3.get_color())
ax_ir.plot(scale_irtf.wave[segment3_indx], np.log10(scale_irtf.flux[segment3_indx])-1, color=l1.get_color())

ax_ir.set_ylim(-17.25,-14.5)
ax_ir.set_xlim(9000, 24900)

annotation_dict_blue = {'CaII 9906': (9686,       -16.15),
                   'MgII 10092': (9857,           -16.15),
                   r'P-$\delta$':(10232,          -16.15),
                   'CI 10691/ HeI 10830': (10500, -16.4),
                   r'P-$\gamma$': (10675,         -16.5),
                   'OI 11290': (11075,            -16.5),
                   'CaII 11836': (11572,          -16.5),
                   'CaII 11947': (11685,          -16.5),
                   r'P-$\beta$':(12503,           -16.5)}

for key in annotation_dict_blue.keys():
    x,y = annotation_dict_blue[key]
    yoffset=0.1
    textoffset = 0.05
    ax_ir.axvline(x, color='k', alpha=0.05, zorder=0)
    ax_ir.plot([x,x], [y, y-yoffset], color='k')
    ax_ir.text(x, y-yoffset-textoffset, key, rotation='vertical', ha='center', va='top',
        fontproperties=font)
    
    
annotation_dict_red = {'SiI 15893': (15632,   -15.25),
                       'CI 16600': (16351,    -15.25), 
                       'CI 16890': (16655,    -15.25),
                       r'Br-$\gamma$':(21480, -15.75),
                       'HeI 20587': (20041,   -15.5)}
for key in annotation_dict_red.keys():
    x,y = annotation_dict_red[key]
    yoffset=0.1
    textoffset = 0.05
    ax_ir.axvline(x, color='k', alpha=0.05, zorder=0)
    ax_ir.plot([x,x], [y, y+yoffset], color='k')
    ax_ir.text(x, y+yoffset+textoffset, key, rotation='vertical', ha='center', va='bottom', 
        fontproperties=font)
ax_ir.set_ylabel('log(Flux)+ offset')
ax_ir.set_xlabel('Wavelength ($\AA$)')
ax_ir.set_yticks([])

ax_ir.text(25000, -16.0, '8d' , ha='left')
ax_ir.text(25000, -16.65, '38d', ha='left')
ax_ir.text(25000, -17.0, '43d', ha='left')
plt.savefig(os.path.join(FIG_DIR, 'ir_spec_montage_log.pdf'))

/Users/bostroem/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:21: RuntimeWarning: divide by zero encountered in log10
/Users/bostroem/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:21: RuntimeWarning: invalid value encountered in log10
/Users/bostroem/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:22: RuntimeWarning: divide by zero encountered in log10
/Users/bostroem/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:23: RuntimeWarning: invalid value encountered in log10


In [383]:
x, y

(3.9861444647105206, -14.455931955649724)